In [5]:
import fastai
import os
import pprint
import torch

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

from sklearn.preprocessing import (
    StandardScaler
)

from typing import (
    Callable,
    Dict,
    List,
    Tuple
)

from Quick.cleaning.loading import (
    examine_dataset,
    remove_infs_and_nans
)

from Quick.cleaning.utils import (
    get_file_path
)

from Quick.runners.deep import (
    run_deep_nn_experiment
)

from Quick.runners.residual import (
    run_residual_deep_nn_experiment
)

from Quick.runners.sk import (
    run_sk_experiment
)

from Quick.runners.torch import (
    run_torch_nn_experiment
)

from rff.layers import (
    GaussianEncoding,
)


from Quick.constants import *

pretty = pprint.PrettyPrinter(indent=4).pprint

In [3]:
use_gpu: bool = False

if(use_gpu):

    if(torch.backends.mps.is_available()): # For Mac M1/M2 chips
        device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
    else: # For NVIDIA cuda chips
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

else:
    device = torch.device('cpu')

print(f'We are currently using the device: {device}')

We are currently using the device: cpu


## We are currently using the CICDDoS-2019 dataset from the Canadian Institute of Cybersecurity, found [here](https://www.unb.ca/cic/datasets/ddos-2019.html). 

Download the csv files in zip form from the website and extract the zip in the ..data directory

In [8]:
ddos_data_path_1: str = '../data/01-12/'
ddos_data_path_2: str = '../data/03-11/'

# Raw datasets, still contain infs/nans/empty and are all primarily 1 class
ddos_data_1: list = [ # https://www.unb.ca/cic/datasets/ddos-2019.html
    'DrDoS_NTP.csv',
    'DrDoS_DNS.csv',
    'DrDoS_LDAP.csv',
    'DrDoS_MSSQL.csv',
    'DrDoS_NetBIOS.csv',
    'DrDoS_SNMP.csv',
    'DrDoS_SSDP.csv',
    'DrDoS_UDP.csv',
    'UDPLag.csv',
    'Syn.csv',
    'TFTP.csv',
]

# Raw datasets, still contain infs/nans/empty and are all primarily 1 class
ddos_data_2: list = [ # https://www.unb.ca/cic/datasets/ddos-2019.html
    'Portmap.csv',
    'NetBIOS.csv',
    'LDAP.csv',
    'MSSQL.csv',
    'UDP.csv',
    'UDPLAG.csv',
    'SYN.csv'
]


# Now we package the datasets and load when necessary
ddos_path_1: callable = get_file_path(ddos_data_path_1)
ddos_path_2: callable = get_file_path(ddos_data_path_2)

ddos_datasets_1: list = list(map(ddos_path_1, ddos_data_1))
ddos_datasets_2: list = list(map(ddos_path_2, ddos_data_2))

print(f'The datasets we will be working with are:')
pretty(ddos_datasets_1)
print('and')
pretty(ddos_datasets_2)

The datasets we will be working with are:
[   '../data/01-12/DrDoS_NTP.csv',
    '../data/01-12/DrDoS_DNS.csv',
    '../data/01-12/DrDoS_LDAP.csv',
    '../data/01-12/DrDoS_MSSQL.csv',
    '../data/01-12/DrDoS_NetBIOS.csv',
    '../data/01-12/DrDoS_SNMP.csv',
    '../data/01-12/DrDoS_SSDP.csv',
    '../data/01-12/DrDoS_UDP.csv',
    '../data/01-12/UDPLag.csv',
    '../data/01-12/Syn.csv',
    '../data/01-12/TFTP.csv']
and
[   '../data/03-11/Portmap.csv',
    '../data/03-11/NetBIOS.csv',
    '../data/03-11/LDAP.csv',
    '../data/03-11/MSSQL.csv',
    '../data/03-11/UDP.csv',
    '../data/03-11/UDPLAG.csv',
    '../data/03-11/SYN.csv']


In [9]:
dataset_1 = examine_dataset(0, ddos_datasets_1.reverse(), ddos_data_1.reverse())

Dataset 0/11: We now look at ../data/01-12/TFTP.csv


Loading Dataset: ../data/01-12/TFTP.csv
	To Dataset Cache: ./cache/TFTP.csv.pickle



/Users/jameshalladay/Programming/TrafficFlowForecasting/prototype/Quick/cleaning/utils.py:49: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filePath, low_memory=True)



        File:				../data/01-12/TFTP.csv  
        Job Number:			0
        Shape:				(20107827, 88)
        Samples:			20107827 
        Features:			88
    
